# Piecewise Efficiency: Heat Pump with Variable COP

## User Story

> *You're installing a heat pump for a commercial building. The heat pump's efficiency (COP - Coefficient of Performance) varies with the outdoor temperature: it's more efficient in mild weather than in cold weather. You want to model this realistically to accurately predict operating costs.*

This notebook introduces:

- **Piecewise linear functions**: Approximate non-linear behavior
- **Variable efficiency**: COP changes with operating conditions
- **LinearConverter with segments**: Multiple operating points
- **Piecewise effects**: Non-linear cost curves

## Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

## The Problem: Variable Heat Pump Efficiency

A heat pump's COP (Coefficient of Performance) depends on the temperature difference between source and sink:

- **Mild weather** (10°C outside): COP ≈ 4.5 (1 kWh electricity → 4.5 kWh heat)
- **Cold weather** (-5°C outside): COP ≈ 2.5 (1 kWh electricity → 2.5 kWh heat)

This non-linear relationship can be approximated using piecewise linear segments.

## Define Time Series Data

In [ ]:
# One winter week
timesteps = pd.date_range('2024-01-22', periods=168, freq='h')
hours = np.arange(168)
hour_of_day = hours % 24

# Outdoor temperature: daily cycle with cold nights
temp_base = 2  # Average temp
temp_amplitude = 5  # Daily variation
outdoor_temp = temp_base + temp_amplitude * np.sin((hour_of_day - 6) * np.pi / 12)

# Add day-to-day variation
np.random.seed(789)
daily_offset = np.repeat(np.random.uniform(-3, 3, 7), 24)
outdoor_temp = outdoor_temp + daily_offset

print(f'Temperature range: {outdoor_temp.min():.1f}°C to {outdoor_temp.max():.1f}°C')

In [ ]:
# Heat demand: inversely related to outdoor temp
# Higher demand when colder
heat_demand = 200 - 8 * outdoor_temp  # Simple linear relationship
heat_demand = np.clip(heat_demand, 100, 300)

print(f'Heat demand range: {heat_demand.min():.0f} - {heat_demand.max():.0f} kW')

In [ ]:
# Visualize with plotly - using xarray and faceting
profiles = xr.Dataset(
    {
        'Outdoor Temp [°C]': xr.DataArray(outdoor_temp, dims=['time'], coords={'time': timesteps}),
        'Heat Demand [kW]': xr.DataArray(heat_demand, dims=['time'], coords={'time': timesteps}),
    }
)

df = profiles.to_dataframe().reset_index().melt(id_vars='time', var_name='variable', value_name='value')
fig = px.line(df, x='time', y='value', facet_col='variable', height=300)
fig.update_yaxes(matches=None, showticklabels=True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))
fig

## Calculate Time-Varying COP

The COP depends on outdoor temperature. We use a simplified Carnot-based formula:

In [ ]:
# COP calculation (simplified)
# Real COP ≈ 0.4 * Carnot COP
T_supply = 45 + 273.15  # Supply temperature 45°C in Kelvin
T_source = outdoor_temp + 273.15  # Outdoor temp in Kelvin

carnot_cop = T_supply / (T_supply - T_source)
real_cop = 0.45 * carnot_cop  # Real efficiency factor
real_cop = np.clip(real_cop, 2.0, 5.0)  # Physical limits

print(f'COP range: {real_cop.min():.2f} - {real_cop.max():.2f}')

In [ ]:
# Plot COP vs temperature using plotly
px.scatter(
    x=outdoor_temp,
    y=real_cop,
    title='Heat Pump COP vs Outdoor Temperature',
    labels={'x': 'Outdoor Temperature [°C]', 'y': 'COP'},
    opacity=0.5,
)

## Approach 1: Simple Model with Time-Varying Efficiency

The simplest approach: use time-varying conversion factors directly.

In [ ]:
flow_system = fx.FlowSystem(timesteps)

flow_system.add_elements(
    fx.Bus('Electricity', carrier='electricity'),
    fx.Bus('Heat', carrier='heat'),
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    # Electricity grid
    fx.Source(
        'Grid',
        outputs=[fx.Flow('Elec', bus='Electricity', size=500, effects_per_flow_hour=0.30)],
    ),
    # Heat pump with time-varying COP
    fx.LinearConverter(
        'HeatPump',
        inputs=[fx.Flow('Elec', bus='Electricity', size=150)],
        outputs=[fx.Flow('Heat', bus='Heat', size=500)],
        # Time-varying conversion factor: Elec * COP = Heat
        conversion_factors=[{'Elec': real_cop, 'Heat': 1}],
    ),
    # Building heat demand
    fx.Sink(
        'Building',
        inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=heat_demand)],
    ),
)

flow_system.optimize(fx.solvers.HighsSolver())
print(f'Total cost: {flow_system.solution["costs"].item():.2f} €')

In [ ]:
flow_system.statistics.plot.balance('Heat')

In [ ]:
flow_system.statistics.plot.balance('Electricity')

## Approach 2: Simple vs Piecewise Efficiency - Model Refinement

Real equipment often has non-linear efficiency curves. Let's compare:
1. **Simple model**: Constant average efficiency
2. **Refined model**: Piecewise linear efficiency that varies with load

This demonstrates how to progressively refine a model for more accurate results.

In [ ]:
# Gas engine efficiency varies with load:
#   - 25% load: 32% efficiency
#   - 50% load: 38% efficiency
#   - 75% load: 42% efficiency (optimal)
#   - 100% load: 40% efficiency

# For the SIMPLE model, we'll use average efficiency (~38%)
SIMPLE_EFFICIENCY = 0.38

# For the PIECEWISE model, we define the actual curve:
# At various operating points (fuel in kW → elec out kW):
# 25% load: 78 kW fuel → 25 kW elec (η=32%)
# 50% load: 132 kW fuel → 50 kW elec (η=38%)
# 75% load: 179 kW fuel → 75 kW elec (η=42%)
# 100% load: 250 kW fuel → 100 kW elec (η=40%)

piecewise_efficiency = fx.PiecewiseConversion(
    {
        'Fuel': fx.Piecewise(
            [
                fx.Piece(start=78, end=132),  # Segment 1: part load
                fx.Piece(start=132, end=179),  # Segment 2: mid load
                fx.Piece(start=179, end=250),  # Segment 3: high load
            ]
        ),
        'Elec': fx.Piecewise(
            [
                fx.Piece(start=25, end=50),  # Segment 1
                fx.Piece(start=50, end=75),  # Segment 2
                fx.Piece(start=75, end=100),  # Segment 3
            ]
        ),
    }
)

print('Piecewise segments (Fuel → Elec):')
fuel_pieces = piecewise_efficiency.piecewises['Fuel'].pieces
elec_pieces = piecewise_efficiency.piecewises['Elec'].pieces
for i, (fuel_piece, elec_piece) in enumerate(zip(fuel_pieces, elec_pieces, strict=False)):
    fuel_avg = (fuel_piece.start + fuel_piece.end) / 2
    elec_avg = (elec_piece.start + elec_piece.end) / 2
    efficiency = elec_avg / fuel_avg * 100
    print(
        f'  Segment {i + 1}: {fuel_piece.start}-{fuel_piece.end} kW fuel → {elec_piece.start}-{elec_piece.end} kW elec (~{efficiency:.1f}% eff)'
    )
print(f'\nSimple model uses constant efficiency: {SIMPLE_EFFICIENCY * 100:.0f}%')

In [ ]:
# Define demand profile
timesteps_simple = pd.date_range('2024-01-22', periods=48, freq='h')  # 2 days
elec_demand_simple = np.concatenate(
    [
        np.linspace(30, 90, 24),  # Day 1: ramp up
        np.linspace(90, 30, 24),  # Day 2: ramp down
    ]
)

# ============================================
# MODEL 1: Simple constant efficiency
# ============================================
fs_simple = fx.FlowSystem(timesteps_simple)

fs_simple.add_elements(
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)]),
    # Engine with SIMPLE constant efficiency
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas', size=300)],
        outputs=[fx.Flow('Elec', bus='Electricity', size=100)],
        conversion_factors=[{'Fuel': 1, 'Elec': SIMPLE_EFFICIENCY}],  # constant 38% efficiency
    ),
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=elec_demand_simple)]),
)

fs_simple.optimize(fx.solvers.HighsSolver())
cost_simple = fs_simple.solution['costs'].item()
print(f'Simple model cost: {cost_simple:.2f} €')

In [ ]:
# ============================================
# MODEL 2: Piecewise efficiency (refined model)
# ============================================
fs_piecewise = fx.FlowSystem(timesteps_simple)

fs_piecewise.add_elements(
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    fx.Source('GasGrid', outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)]),
    # Engine with PIECEWISE efficiency (load-dependent)
    fx.LinearConverter(
        'GasEngine',
        inputs=[fx.Flow('Fuel', bus='Gas')],
        outputs=[fx.Flow('Elec', bus='Electricity')],
        piecewise_conversion=piecewise_efficiency,
    ),
    fx.Sink('Load', inputs=[fx.Flow('Elec', bus='Electricity', size=1, fixed_relative_profile=elec_demand_simple)]),
)

fs_piecewise.optimize(fx.solvers.HighsSolver())
cost_piecewise = fs_piecewise.solution['costs'].item()
print(f'Piecewise model cost: {cost_piecewise:.2f} €')

In [ ]:
# Visualize the piecewise conversion curve
fs_piecewise.components['GasEngine'].piecewise_conversion.plot(
    x_flow='Fuel', title='Gas Engine: Fuel Input vs Electricity Output'
)

In [ ]:
# ============================================
# Compare Results: Simple vs Piecewise
# ============================================
print('=== Efficiency Model Comparison ===')
print(f'Simple model (constant {SIMPLE_EFFICIENCY * 100:.0f}% eff):  {cost_simple:.2f} €')
print(f'Piecewise model (load-dependent eff): {cost_piecewise:.2f} €')
print(f'Difference: {cost_piecewise - cost_simple:.2f} € ({(cost_piecewise - cost_simple) / cost_simple * 100:+.1f}%)')

# Compare fuel consumption
fuel_simple = fs_simple.statistics.flow_rates['GasEngine(Fuel)'].sum().item()
fuel_piecewise = fs_piecewise.statistics.flow_rates['GasEngine(Fuel)'].sum().item()
print('\nTotal fuel consumption:')
print(f'  Simple:    {fuel_simple:.1f} kWh')
print(f'  Piecewise: {fuel_piecewise:.1f} kWh')
print(
    f'  Difference: {fuel_piecewise - fuel_simple:.1f} kWh ({(fuel_piecewise - fuel_simple) / fuel_simple * 100:+.1f}%)'
)

## Approach 3: Simple vs Piecewise Investment Costs

Investment costs often have economies of scale - larger systems cost less per unit. Let's compare:
1. **Simple model**: Constant cost per kWh (average: 800 €/kWh)
2. **Refined model**: Piecewise costs that decrease with size

In [ ]:
fs_piecewise.statistics.plot.balance('Gas')

In [ ]:
# Investment cost curve with economies of scale:
# These represent the daily cost allocation of storage investment
#
# For SIMPLE model: constant marginal cost
SIMPLE_INVEST_COST = 0.20  # €/kWh (daily amortized)

# For PIECEWISE model: decreasing marginal cost (economies of scale)
# - Small storage (0-100 kWh): 0.20 €/kWh - same as simple
# - Medium storage (100-300 kWh): 0.15 €/kWh - better value
# - Large storage (300-600 kWh): 0.10 €/kWh - bulk discount

piecewise_invest_costs = fx.PiecewiseEffects(
    piecewise_origin=fx.Piecewise(
        [
            fx.Piece(start=0, end=100),
            fx.Piece(start=100, end=300),
            fx.Piece(start=300, end=600),
        ]
    ),
    piecewise_shares={
        'costs': fx.Piecewise(
            [
                fx.Piece(start=0, end=20),  # 0.20 €/kWh
                fx.Piece(start=20, end=50),  # 0.15 €/kWh
                fx.Piece(start=50, end=80),  # 0.10 €/kWh
            ]
        )
    },
)

print('Investment cost structure (daily amortized):')
print(f'  Simple model: constant {SIMPLE_INVEST_COST:.2f} €/kWh')
print('  Piecewise model (economies of scale):')
print('    0-100 kWh:   0.20 €/kWh')
print('    100-300 kWh: 0.15 €/kWh')
print('    300-600 kWh: 0.10 €/kWh')

In [ ]:
# Create base system for storage investment comparison
# Scenario: Price arbitrage - charge when cheap, discharge when expensive
# 3-day horizon to make storage investment meaningful

timesteps_invest = pd.date_range('2024-01-22', periods=72, freq='h')

# Steady demand pattern repeated over 3 days
heat_demand_invest = np.tile(
    np.concatenate(
        [
            np.full(8, 100),  # Night
            np.full(4, 120),  # Morning
            np.full(4, 110),  # Midday
            np.full(4, 130),  # Evening peak
            np.full(4, 105),  # Late evening
        ]
    ),
    3,
)

# Strong price signal for arbitrage
energy_price_invest = np.tile(
    np.concatenate(
        [
            np.full(8, 0.08),  # Night: cheap - perfect for charging
            np.full(4, 0.20),  # Morning: expensive
            np.full(4, 0.12),  # Midday: medium
            np.full(4, 0.25),  # Evening: very expensive - best time to discharge
            np.full(4, 0.10),  # Late evening
        ]
    ),
    3,
)

fs_base = fx.FlowSystem(timesteps_invest)

fs_base.add_elements(
    fx.Bus('Heat', carrier='heat'),
    fx.Effect('costs', '€', 'Costs', is_standard=True, is_objective=True),
    fx.Source('HeatSource', outputs=[fx.Flow('Heat', bus='Heat', size=300, effects_per_flow_hour=energy_price_invest)]),
    fx.Sink('HeatSink', inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=heat_demand_invest)]),
)

print('Time horizon: 72 hours (3 days)')
print(f'Demand range: {heat_demand_invest.min():.0f} - {heat_demand_invest.max():.0f} kW')
print(f'Price range: {energy_price_invest.min():.2f} - {energy_price_invest.max():.2f} €/kWh')
print(f'Price spread: {energy_price_invest.max() - energy_price_invest.min():.2f} €/kWh')
print('=> Storage value comes from charging cheap, discharging expensive')

In [ ]:
# ============================================
# MODEL 1: Simple linear investment costs
# ============================================
fs_simple_invest = fs_base.copy()

fs_simple_invest.add_elements(
    fx.Storage(
        'ThermalStorage',
        charging=fx.Flow('charge', bus='Heat', size=200),
        discharging=fx.Flow('discharge', bus='Heat', size=200),
        capacity_in_flow_hours=fx.InvestParameters(
            effects_of_investment_per_size=SIMPLE_INVEST_COST,  # constant 0.20 €/kWh
            minimum_size=0,
            maximum_size=600,
        ),
        initial_charge_state=0,
        eta_charge=0.95,
        eta_discharge=0.95,
        relative_loss_per_hour=0.005,
    ),
)

fs_simple_invest.optimize(fx.solvers.HighsSolver())
cost_simple_invest = fs_simple_invest.solution['costs'].item()
size_simple = fs_simple_invest.solution['ThermalStorage|size'].item()
print('Simple investment model:')
print(f'  Total cost: {cost_simple_invest:,.2f} €')
print(f'  Storage size: {size_simple:.1f} kWh')

In [ ]:
# ============================================
# MODEL 2: Piecewise investment costs (economies of scale)
# ============================================
fs_piecewise_invest = fs_base.copy()

fs_piecewise_invest.add_elements(
    fx.Storage(
        'ThermalStorage',
        charging=fx.Flow('charge', bus='Heat', size=200),
        discharging=fx.Flow('discharge', bus='Heat', size=200),
        capacity_in_flow_hours=fx.InvestParameters(
            piecewise_effects_of_investment=piecewise_invest_costs,  # economies of scale
            minimum_size=0,
            maximum_size=600,
        ),
        initial_charge_state=0,
        eta_charge=0.95,
        eta_discharge=0.95,
        relative_loss_per_hour=0.005,
    ),
)

fs_piecewise_invest.optimize(fx.solvers.HighsSolver())
cost_piecewise_invest = fs_piecewise_invest.solution['costs'].item()
size_piecewise = fs_piecewise_invest.solution['ThermalStorage|size'].item()
print('Piecewise investment model:')
print(f'  Total cost: {cost_piecewise_invest:,.2f} €')
print(f'  Storage size: {size_piecewise:.1f} kWh')

In [ ]:
# Visualize the piecewise investment cost curve
piecewise_invest_costs.plot(title='Storage Investment Costs (Economies of Scale)')

In [ ]:
# ============================================
# Compare Results: Simple vs Piecewise Investment
# ============================================

# Calculate operating cost baseline (without storage) - suppress solver output
import contextlib
import os

with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
        fs_no_storage = fs_base.copy()
        fs_no_storage.optimize(fx.solvers.HighsSolver())
        cost_baseline = fs_no_storage.solution['costs'].item()

print('=' * 60)
print('INVESTMENT MODEL COMPARISON')
print('=' * 60)

print(f'\nBaseline (no storage): {cost_baseline:.2f} €')

print(f'\nSimple model (constant {SIMPLE_INVEST_COST:.2f} €/kWh):')
print(f'  Storage size: {size_simple:.1f} kWh')
print(f'  Total cost:   {cost_simple_invest:,.2f} €')
invest_simple = size_simple * SIMPLE_INVEST_COST
print(f'  - Investment: {invest_simple:.2f} € ({size_simple:.0f} kWh × {SIMPLE_INVEST_COST:.2f} €/kWh)')
print(f'  - Operating:  {cost_simple_invest - invest_simple:.2f} €')
print(f'  - Savings:    {cost_baseline - cost_simple_invest:.2f} € vs no storage')

print('\nPiecewise model (economies of scale):')
print(f'  Storage size: {size_piecewise:.1f} kWh')
print(f'  Total cost:   {cost_piecewise_invest:,.2f} €')

# Calculate piecewise investment cost
if size_piecewise <= 100:
    invest_piecewise = size_piecewise * 0.20
elif size_piecewise <= 300:
    invest_piecewise = 20 + (size_piecewise - 100) * 0.15
else:
    invest_piecewise = 50 + (size_piecewise - 300) * 0.10

avg_cost = invest_piecewise / size_piecewise if size_piecewise > 0 else 0
print(f'  - Investment: {invest_piecewise:.2f} € (avg: {avg_cost:.2f} €/kWh)')
print(f'  - Operating:  {cost_piecewise_invest - invest_piecewise:.2f} €')
print(f'  - Savings:    {cost_baseline - cost_piecewise_invest:.2f} € vs no storage')

print('\n' + '-' * 60)
print('KEY INSIGHT:')
if size_piecewise > size_simple:
    print(f'  Economies of scale result in {size_piecewise - size_simple:.0f} kWh MORE storage')
    print(f'  with {cost_simple_invest - cost_piecewise_invest:.2f} € LOWER total cost.')
else:
    print(f'  Both models choose similar storage size (~{size_simple:.0f} kWh)')
    print(f'  but piecewise saves {cost_simple_invest - cost_piecewise_invest:.2f} € through economies of scale.')
    print(f'  Average cost: {SIMPLE_INVEST_COST:.2f} €/kWh (simple) vs {avg_cost:.2f} €/kWh (piecewise)')

### Energy Flow Sankey (Heat Pump System)

A Sankey diagram for the heat pump system:

In [ ]:
fs_piecewise_invest.statistics.plot.sankey()

<cell_type>markdown</cell_type>## Key Concepts

### Time-Varying Conversion Factors

For temperature-dependent efficiency, use arrays with flow labels as keys:
```python
fx.LinearConverter(
    'HeatPump',
    inputs=[fx.Flow('Elec', bus='Electricity', size=150)],
    outputs=[fx.Flow('Heat', bus='Heat', size=500)],
    # Equation: Elec * COP = Heat * 1
    conversion_factors=[{'Elec': cop_array, 'Heat': 1}],
)
```

### Piecewise Linear Functions

For non-linear relationships (part-load efficiency), use `PiecewiseConversion`:
```python
fx.PiecewiseConversion({
    'Fuel': fx.Piecewise([
        fx.Piece(start=78, end=132),   # Segment 1
        fx.Piece(start=132, end=179),  # Segment 2
    ]),
    'Elec': fx.Piecewise([
        fx.Piece(start=25, end=50),    # Segment 1
        fx.Piece(start=50, end=75),    # Segment 2
    ]),
})
```

### When to Use Each Approach

| Approach | Use Case | Complexity |
|----------|----------|------------|
| Time-varying factors | Efficiency depends on external conditions (temp, price) | Low |
| PiecewiseConversion | Efficiency depends on load level | Medium |
| PiecewiseEffects | Non-linear costs (economies of scale) | Medium |

## Summary

You learned how to:

- Model **time-varying efficiency** using conversion factor arrays
- Use **PiecewiseConversion** for load-dependent efficiency curves
- Apply **PiecewiseEffects** for non-linear cost functions
- Choose the right approach for your modeling needs

### Next Steps

- **[07-scenarios-and-periods](07-scenarios-and-periods.ipynb)**: Multi-scenario planning
- **[08-large-scale-optimization](08-large-scale-optimization.ipynb)**: Computational efficiency techniques